# Tests on the RAG injection and queries asks

## libraries

In [1]:
from groq import Groq
import os 
#from pydub import AudioSegment
import subprocess
import os
from loguru import logger
from pyannote.audio import Pipeline 
from google.cloud import speech_v2
import time
import torch

/home/jeanlsoto54/.pyenv/versions/cablewatch/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jeanlsoto54/.pyenv/versions/cablewatch/lib/python3.13/site-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()


In [ ]:
#!pip install pyannote.audio


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


## Process of transcription

In [ ]:
# function to open the ffmpeg and convert the ts into mp3 
def convert_ts_to_mp3_direct(ts_file):
    mp3_file = ts_file.replace('.ts', '.mp3')
    command = [
        "ffmpeg", "-i", ts_file, 
        "-vn", 
        "-acodec", "libmp3lame", 
        "-y", 
        mp3_file
    ]
    subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    return mp3_file

audio_test ="/home/jeanlsoto54/code/jeanluissoto/project_final/cablewatch/data/ingest/segment_2025-12-24_16h36mn50.ts"
convert_ts_to_mp3_direct(audio_test)

'/home/jeanlsoto54/code/jeanluissoto/project_final/cablewatch/data/ingest/segment_2025-12-24_16h36mn50.mp3'

In [2]:
import getpass
groq_api_key = getpass.getpass('API key here:')

In [3]:
filename = '/home/jeanlsoto54/code/jeanluissoto/project_final/cablewatch/data/ingest/segment_2025-12-24_16h36mn50.mp3'

Méthode OpenAI, modèle : Whisper v3 (avec une inférence exécutée sur Groq, pour la vitesse et la puissance de ce modèle open source)

In [19]:

def transcription_groqwhisper(api_key, filename):
  inicio = time.time()
  client = Groq(api_key=api_key)
  

  with open(filename, "rb") as file:
      transcription = client.audio.transcriptions.create(
        file=(filename, file.read()),
        model="whisper-large-v3-turbo",
        temperature=0,
        response_format="verbose_json",
        language='fr',
        timestamp_granularities=['segment']
      )
  fin = time.time()
  logger.info(f"Transcription completed in {fin - inicio:.2f} seconds")
  return transcription

In [20]:
transcription = transcription_groqwhisper(groq_api_key,filename)

2025-12-25 21:41:00.575 | INFO     | __main__:transcription_groqwhisper:16 - Transcription completed in 1.14 seconds


In [21]:
transcription.text

" Cornu. Bonsoir. Jusqu'à 7 millions de téléspectateurs suivent cette interview. Un record historique. Qui se présente devant nous ce soir ? Est-ce l'ex-premier ministre ? Le futur premier ministre ? Un médiateur ? Qui ? Le premier ministre qui a démissionné, qui a remis la démission du gouvernement lundi matin, donc j'expédie les affaires courantes. Il est vrai que depuis 48 heures, le président de la République m'a fait revenir lundi après-midi, considérant que les trois semaines que j'avais passées dans le secret, parfois ou plus ou moins publiquement avec les différents"

GCP Modele: Speech-to-Text (V2) solution avec couts

In [ ]:
from google.cloud import speech_v2

client = speech_v2.SpeechClient(client_options={"api_endpoint": "europe-west4-speech.googleapis.com"})

create_request = speech_v2.types.CreateRecognizerRequest(
    parent="projects/teak-instrument-480811-u5/locations/europe-west4",
    recognizer_id="my-diarization-recognizer",  # HYPHEN not underscore
    recognizer=speech_v2.types.Recognizer(
        display_name="French Diarization Recognizer",
        model="latest_long",
        default_recognition_config=speech_v2.types.RecognitionConfig(
            auto_decoding_config={},
            language_codes=["fr-FR"],
            model="latest_long",
            features=speech_v2.types.RecognitionFeatures(
                enable_automatic_punctuation=True,
            ),
        ),
    ),
)
recognizer = client.create_recognizer(request=create_request)
print(f"Created: {recognizer}")


AttributeError: 'Operation' object has no attribute 'name'

In [69]:
client = speech_v2.SpeechClient(client_options={"api_endpoint": "europe-west4-speech.googleapis.com"})
parent = "projects/teak-instrument-480811-u5/locations/europe-west4"
recognizers = client.list_recognizers(parent=parent)
for r in recognizers:
    print(f"ID: {r.name}, Model: {r.model}, Supports Diarization: {hasattr(r.default_recognition_config.features, 'diarization_config')}")


ID: projects/910491225982/locations/europe-west4/recognizers/my-diarization-recognizer, Model: latest_long, Supports Diarization: True


In [ ]:
def transcription_gcp(
        filename
):
    

    project_id = "teak-instrument-480811-u5"
    location = "europe-west4"  
    recognizer_id = "my-diarization-recognizer"  # Your custom recognizer
    client_options = {"api_endpoint": f"{location}-speech.googleapis.com"}
    recognizer_path = f"projects/{project_id}/locations/{location}/recognizers/{recognizer_id}"
    
    client = speech_v2.SpeechClient(client_options=client_options)
    
    config = speech_v2.types.RecognitionConfig(
        auto_decoding_config={},
        language_codes=["fr-FR"],
        # features=speech_v2.types.RecognitionFeatures(
        #     diarization_config=speech_v2.types.SpeakerDiarizationConfig(
        #         min_speaker_count=1,
        #         max_speaker_count=5,)
            
        # )
    )

    inicio = time.time()
    with open(filename, 'rb') as f:
        content = f.read()
    
    request = speech_v2.types.RecognizeRequest(
        recognizer=recognizer_path,
        config=config,
        content=content,
    )
    response = client.recognize(request=request)
    logger.info(response)

    billed_duration = response.metadata.total_billed_duration.seconds

    script_final = []
    current_speaker = None
    current_sentence = ""

    for result in response.results:
        for word_info in result.alternatives[0].words:
            speaker = word_info.speaker_label or "Unknown"
            word = word_info.word

            if speaker != current_speaker:
                if current_speaker is not None:
                    script_final.append(f"Locuteur {current_speaker}: {current_sentence.strip()}")
                current_speaker = speaker
                current_sentence = word + " "
            else:
                current_sentence += word + " "

    if current_sentence:
        script_final.append(f"Locuteur {current_speaker}: {current_sentence.strip()}")
    
    fin = time.time()
    logger.info(f"Transcription completed in {fin - inicio:.2f} seconds")
    return script_final, billed_duration


In [83]:
def transcription_gcp(
        filename
):
    

    project_id = "teak-instrument-480811-u5"
    location = "eu"  
    recognizer_id = "_"  # Your custom recognizer
    client_options = {"api_endpoint": f"{location}-speech.googleapis.com"}
    recognizer_path = f"projects/{project_id}/locations/{location}/recognizers/{recognizer_id}"
    
    client = speech_v2.SpeechClient(client_options=client_options)
    
    config = speech_v2.types.RecognitionConfig(
        auto_decoding_config={},
        language_codes=["fr-FR"],
        model="chirp_3",
        features=speech_v2.types.RecognitionFeatures(
            diarization_config=speech_v2.types.SpeakerDiarizationConfig(
                min_speaker_count=1,
                max_speaker_count=5,)
            
        )
    )

    inicio = time.time()
    with open(filename, 'rb') as f:
        content = f.read()
    
    request = speech_v2.types.RecognizeRequest(
        recognizer=recognizer_path,
        config=config,
        content=content,
    )
    response = client.recognize(request=request)
    logger.info(response)

    billed_duration = response.metadata.total_billed_duration.seconds

    script_final = []
    current_speaker = None
    current_sentence = ""

    for result in response.results:
        for word_info in result.alternatives[0].words:
            speaker = word_info.speaker_label or "Unknown"
            word = word_info.word

            if speaker != current_speaker:
                if current_speaker is not None:
                    script_final.append(f"Locuteur {current_speaker}: {current_sentence.strip()}")
                current_speaker = speaker
                current_sentence = word + " "
            else:
                current_sentence += word + " "

    if current_sentence:
        script_final.append(f"Locuteur {current_speaker}: {current_sentence.strip()}")
    
    fin = time.time()
    logger.info(f"Transcription completed in {fin - inicio:.2f} seconds")
    return script_final, billed_duration


In [84]:
script_final, billed_duration= transcription_gcp(filename)

2025-12-25 23:07:38.389 | INFO     | __main__:transcription_gcp:36 - metadata {
  total_billed_duration {
    seconds: 31
  }
  request_id: "82e569a3-0000-2ed2-8511-582429ce8efc"
}
results {
  alternatives {
    transcript: "Cornu. Bonsoir. Jusqu\'à 7 millions de téléspectateurs suivent cette interview, un record historique. Qui se présente devant nous ce soir? Est-ce l\'ex-Premier ministre, le futur Premier ministre, un médiateur? Qui? Bah, le Premier ministre qui a démissionné, qui a remis la démission du gouvernement lundi matin, donc j\'expédie les affaires courantes. Il est vrai que depuis 48h, le Président de la République m\'a fait revenir lundi après-midi, considérant que les 3 semaines que j\'avais passées dans le secret, parfois, ou plus ou moins public publiquement avec les"
    words {
      word: "Cornu."
      speaker_label: "0"
    }
    words {
      word: "Bonsoir."
      speaker_label: "1"
    }
    words {
      word: "Jusqu\'à"
      speaker_label: "2"
    }
    wor

In [85]:
script_final

['Locuteur 0: Cornu.',
 'Locuteur 1: Bonsoir.',
 "Locuteur 2: Jusqu'à 7 millions de téléspectateurs suivent cette interview, un record historique.",
 "Locuteur 0: Qui se présente devant nous ce soir? Est-ce l'ex-Premier ministre, le futur Premier ministre, un médiateur? Qui?",
 "Locuteur 1: Bah, le Premier ministre qui a démissionné, qui a remis la démission du gouvernement lundi matin, donc j'expédie les affaires courantes. Il est vrai que depuis 48h, le Président de la République m'a fait revenir lundi après-midi, considérant que les 3 semaines que j'avais passées dans le secret, parfois, ou plus ou moins public publiquement avec les"]

In [86]:
billed_duration

31

Option Open Source: (Hugging Face/ pyannote)

In [6]:

hf_api_key = getpass.getpass('Hugging Face API key here:')

In [89]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
def transcription_pyannote(filename,hf_api,groq_key): #
    logger.info("Démarrage de la diarisation (Pyannote)...")
    pipeline = Pipeline.from_pretrained(
        "pyannote/speaker-diarization-3.1", 
        use_auth_token=hf_api
    )
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    pipeline.to(device)

    diarization = pipeline(filename)
    logger.info("Diarisation terminée.")

    logger.info("Démarrage de la transcription (Groq)...")
    client = Groq(api_key=groq_key)
    
    inicio_total = time.time()
    with open(filename, "rb") as file:
        transcription = client.audio.transcriptions.create(
            file=(filename, file.read()),
            model="whisper-large-v3-turbo",
            temperature=0,
            response_format="verbose_json",
            language='fr'
        )
    
    logger.info("Fusion des résultats et création du script...")
    
    script_final = []
    dernier_locuteur = None
    bloc_texte = ""

  
    for seg in transcription.segments:
        start_seg = seg['start']
        end_seg = seg['end']
        texte = seg['text'].strip()
        
     
        milieu = (start_seg + end_seg) / 2
        locuteur_actuel = "Inconnu"
        
        for turn, _, speaker in diarization.itertracks(yield_label=True):
            if turn.start <= milieu <= turn.end:
                locuteur_actuel = speaker
                break
        
    
        if locuteur_actuel == dernier_locuteur:
            bloc_texte += " " + texte
        else:
  
            if dernier_locuteur is not None:
                script_final.append(f"[{dernier_locuteur}] : {bloc_texte}")
            dernier_locuteur = locuteur_actuel
            bloc_texte = texte

   
    if bloc_texte:
        script_final.append(f"[{dernier_locuteur}] : {bloc_texte}")

      
    fin_total = time.time()
    logger.info(f"Processus terminé en {fin_total - inicio_total:.2f}s.")
    
    return script_final

In [10]:
transcription = transcription_pyannote(filename,hf_api_key,groq_api_key) #

2025-12-26 10:23:34.042 | INFO     | __main__:transcription_pyannote:2 - Démarrage de la diarisation (Pyannote)...
/home/jeanlsoto54/.pyenv/versions/cablewatch/lib/python3.13/site-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()



Could not download 'pyannote/segmentation-3.0' model.
It might be because the model is private or gated so make
sure to authenticate. Visit https://hf.co/settings/tokens to
create your access token and retry with:

   >>> Model.from_pretrained('pyannote/segmentation-3.0',
   ...                       use_auth_token=YOUR_AUTH_TOKEN)

If this still does not work, it might be because the model is gated:
visit https://hf.co/pyannote/segmentation-3.0 to accept the user conditions.


AttributeError: 'NoneType' object has no attribute 'eval'

For the enterprise option in here: https://dashboard.pyannote.ai/signin


Wisper X 
https://aicloudautomation.net/projects/whisperx/

In [99]:
pip install git+https://github.com/m-bain/whisperx.git

  Cloning https://github.com/m-bain/whisperx.git to /tmp/pip-req-build-5r8wrwjl
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperx.git /tmp/pip-req-build-5r8wrwjl
  Resolved https://github.com/m-bain/whisperx.git to commit d32ec3e3012ec4c0934f4088424c32f3f038b249
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached semver-3.0.4-py3-none-any.whl.metadata (6.8 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.6/39.6 MB 7.8 MB/s  0:00:05m0:00:0100:01
   ━━

In [101]:
import whisperx
import gc

device = "cuda" 
audio_file = filename
batch_size = 16 
compute_type = "float16"

model = whisperx.load_model("large-v2", device, compute_type=compute_type)
audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size)

model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

diarize_model = whisperx.DiarizationPipeline(use_auth_token=hf_api_key, device=device)
diarize_segments = diarize_model(audio)
result = whisperx.assign_word_speakers(diarize_segments, result)

# 4. Print results
for segment in result["segments"]:
    print(f"[{segment['speaker']}] {segment['start']:.2f}s: {segment['text']}")

2025-12-26 00:43:15 - whisperx.asr - INFO - No language specified, language will be detected for each audio file (increases inference time)
2025-12-26 00:43:15 - whisperx.vads.pyannote - INFO - Performing voice activity detection using Pyannote...


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.6.0. To apply the upgrade to your files permanently, run `python -m lightning.pytorch.utilities.upgrade_checkpoint ../../../../../.pyenv/versions/cablewatch/lib/python3.13/site-packages/whisperx/assets/pytorch_model.bin`


TypeError: Inference.__init__() got an unexpected keyword argument 'use_auth_token'

Nemo Envidia


In [ ]:
from nemo.collections.asr.models import ASRModel
from nemo.collections.asr.models.speaker_models import SpkrEmbeddingLabelModel

# Load pretrained speaker diarization model
model = SpkrEmbeddingLabelModel.from_pretrained("nvidia/speakerverification_en_titanet_large")


In [ ]:
from simple_diarizer.diarizer import Diarizer

diarizer = Diarizer()
segments = diarizer.diarize("audio.wav", num_speakers=2)
